In [ ]:
inputPath = "/databricks-datasets/structured-streaming/events/"
jsonSchema = StructType([
    StructField('time', TimestampType(), True),
    StructField('action', StringType(), True ),
])
streamingInputDF = ( spark
    .readStream
    .schema(jsonSchema)
    .option('maxFilesPerTrigger', 1)
    .json(inputPath)
)
streamingCountsDF = ( streamingInputDF
    .groupBy(
        streamInputDF.action,
        window(streamingInputDF.time, '1 hour')
    ).count()
)

In [ ]:
query = ( 
    streamingCountsDF.writeStream
    .format('memory')
    .queryName('counts')
    .outputMode('complete')
    .start()
)

In [ ]:
%sql
select action, 
    date_format(window.end, 'MMM-dd HH:mm') as time,
    count
from counts
order by time, action